In [33]:
import yfinance as yf
import pandas as pd
import numpy as np
import json
import requests

In [26]:
# Retrieve Yahoo! Finance Sustainability Scores for each ticker up to 6 companies
tickers=['AAPL', 'TSLA','RIO.L', 'TMO','XRX', 'R', 'NVDA','SBUX','TLRY','MYOR','BNII', 'PGIS', 'MS', 'JNJ', 'FB','JPM' 'BA', 'BRKB', 'MO',  'QCOMM', 'TXN', 'SNBR', 'AZN', 'HSBA','GOGL', 'JNJ', 'FAPA']

def get_sustainability_score(tickers):
    result = pd.DataFrame()
    
    for ticker in tickers:
        print("TRYING FOR THIS TICKER", ticker)
        i_y = yf.Ticker(ticker)
        try:
            if i_y.sustainability is not None and len(result.index) < 7:
                temp = pd.DataFrame.transpose(i_y.sustainability)
                temp['Ticker'] = str(i_y.ticker)
                result = result.append(temp)
                print(len(result.index))
            elif len(result.index) == 6:
                break
        except IndexError:
            pass
        except KeyError:
            pass
      
    return result


In [27]:
scores=get_sustainability_score(tickers)

TRYING FOR THIS TICKER AAPL
1
TRYING FOR THIS TICKER TSLA
2
TRYING FOR THIS TICKER RIO.L
3
TRYING FOR THIS TICKER TMO
4
TRYING FOR THIS TICKER XRX
TRYING FOR THIS TICKER R
TRYING FOR THIS TICKER NVDA
5
TRYING FOR THIS TICKER SBUX
6
TRYING FOR THIS TICKER TLRY


In [28]:
#testing result
scores

2021-3,palmOil,controversialWeapons,gambling,socialScore,nuclear,furLeather,alcoholic,gmo,catholic,socialPercentile,...,coal,pesticides,adult,percentile,peerGroup,smallArms,environmentScore,governancePercentile,militaryContract,Ticker
Value,False,False,False,7.69,False,False,False,False,False,0,...,False,False,False,11.77,Technology Hardware,False,0.1,0,False,AAPL
Value,False,False,False,17.31,False,False,False,False,False,0,...,False,False,False,61.89,Automobiles,False,2.95,0,False,TSLA
Value,False,False,False,12.38,False,False,False,False,False,0,...,True,False,False,66.46,Diversified Metals,False,12.99,0,False,RIO.L
Value,False,False,False,6.03,False,False,False,False,True,0,...,False,False,False,6.06,Pharmaceuticals,False,1.52,0,False,TMO
Value,False,False,False,4.15,False,False,False,False,False,0,...,False,False,False,4.83,Semiconductors,False,2.33,0,False,NVDA
Value,False,False,False,13.53,False,False,False,False,False,0,...,False,False,False,23.72,Consumer Services,False,2.59,0,False,SBUX


In [29]:
#get fin data for that df of scores
# API KEY
token = "&token=c0j6hdv48v6tlon085cg"

def financial_data(scores):
    # Getting the CompanyProfile2 endpoint from finnhub and taking certain ones out
    responses1 = list()

    for ticker in scores.Ticker:
        r1 = requests.get('https://finnhub.io/api/v1/stock/profile2?symbol={}'.format(ticker) + token)
        data1 = json.loads(r1.text)
        responses1.append(data1)

    df1 = pd.DataFrame.from_dict(responses1)
    df1 = df1.drop(columns=['phone', 'logo'])
    df1 = df1.rename(columns={'country':'Country', 'currency':'Currency', 'exchange':'Exchange',
                              'finnhubIndustry':'Industry', 'ipo':'IPO', 'marketCapitalization':'Market Capitalization',
                              'name':'Name', 'shareOutstanding':'Share Outstanding', 'ticker':'Ticker', 'weburl':'URL'})


     #only select CompanyNewsScore from the sentiment score endpoint

    responses3 = list()

    for ticker in scores.Ticker:
        r3 = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol={}'.format(ticker) + token)
        data3 = json.loads(r3.text)
        responses3.append(data3)


    df3 = pd.DataFrame.from_dict(responses3)
    df3 = pd.DataFrame(df3['companyNewsScore'])
    df3 = df3.rename(columns={'companyNewsScore':'Company News Score'})

    # select current stock price from the quote endpoint

    responses4 = list()

    for ticker in scores.Ticker:
        r4 = requests.get('https://finnhub.io/api/v1/quote?symbol={}'.format(ticker) + token)
        data4 = json.loads(r4.text)
        responses4.append(data4)

    df4 = pd.DataFrame.from_dict(responses4)
    df4 = df4.drop(columns=['h', 'l', 'o', 'pc', 't'])
    df4 = df4.rename(columns={'c': 'Current Stock Price'})

    # adding everything together, rearranging the columns

    finaldf = pd.concat([df1, df3, df4], axis=1)
    finaldf = finaldf[['Ticker', 'Name', 'Country', 'Currency', 'Exchange', 'Industry', 'IPO', 'Current Stock Price', 'Market Capitalization', 'Share Outstanding', 'Company News Score', 'URL']]
    return finaldf


In [30]:
fin=financial_data(scores)

In [32]:
fin #there we go

,Ticker,Name,Country,Currency,Exchange,Industry,IPO,Current Stock Price,Market Capitalization,Share Outstanding,Company News Score,URL
0,AAPL,Apple Inc,US,USD,NASDAQ NMS - GLOBAL MARKET,Technology,1980-12-12,121.085,1953463.00,16788.096000,0.4440,https://www.apple.com/
1,TSLA,Tesla Inc,US,USD,NASDAQ NMS - GLOBAL MARKET,Automobiles,2010-06-29,673.580,540397.50,959.853504,0.7046,https://www.tesla.com/
2,RIO.L,Rio Tinto PLC,GB,USD,LONDON STOCK EXCHANGE,Metals & Mining,1973-11-01,NaN,97327.94,1618.898121,NaN,http://www.riotinto.com/
3,TMO,Thermo Fisher Scientific Inc,US,USD,"NEW YORK STOCK EXCHANGE, INC.",Life Sciences Tools & Services,1980-08-27,448.110,173210.00,396.335191,0.5000,https://www.thermofisher.com/
4,NVDA,NVIDIA Corp,US,USD,NASDAQ NMS - GLOBAL MARKET,Semiconductors,1999-01-22,500.810,287512.60,620.000000,0.9500,https://www.nvidia.com/
5,SBUX,Starbucks Corp,US,USD,NASDAQ NMS - GLOBAL MARKET,"Hotels, Restaurants & Leisure",1992-06-01,106.550,123651.80,1177.300000,0.7220,https://www.starbucks.com/
